In [ ]:
# Check if get_enso_composites hasn't been run - if not, run to get enso variables
if plot_enso_composites == 0:
    %run ./get_enso_composites.ipynb

In [ ]:
# Load CCF kernel from Ryan Scott's github
kernel = xr.open_dataset('/data/keeling/a/tjhanke2/enso-energy-budget/Data/raw_data/CRE_kernel_data/Scott_Myers_meteorological_kernels_CERES-FBCT_ERA5.nc')

# Shift lon from [-180,180] to [0,360]
kernel_rolled = kernel.assign_coords(longitude=((kernel.longitude % 360) - 2.5)).roll(longitude=(kernel.dims['longitude'] // 2))

In [ ]:
# Downsample the CCF composites
SST_en_loRes   = SST_enso / kernel_rolled.sigma_xi.sel(i=1.0)
EIS_en_loRes   = EIS_enso / kernel_rolled.sigma_xi.sel(i=2.0)
Tadv_en_loRes  = Tadv_enso / kernel_rolled.sigma_xi.sel(i=3.0)
RH700_en_loRes = RH700_enso / kernel_rolled.sigma_xi.sel(i=4.0)
w700_en_loRes  = w700_enso / kernel_rolled.sigma_xi.sel(i=5.0)
ws_en_loRes    = ws_enso / kernel_rolled.sigma_xi.sel(i=6.0)

In [ ]:
# Separate kernels into dR sensitivities
dR_dSST   = kernel_rolled.dRdxi.sel(i=1.0) # dR/dSST
dR_dEIS   = kernel_rolled.dRdxi.sel(i=2.0) # dR/dEIS
dR_dTadv  = kernel_rolled.dRdxi.sel(i=3.0) # dR/dTadv
dR_dRH700 = kernel_rolled.dRdxi.sel(i=4.0) # dR/dRH700
dR_dw700  = kernel_rolled.dRdxi.sel(i=5.0) # dR/dw700
dR_dws    = kernel_rolled.dRdxi.sel(i=6.0) # dR/dws

In [ ]:
# Now get low cloud radiative effect
CRE_SST   = dR_dSST * SST_en_loRes
CRE_EIS   = dR_dEIS * EIS_en_loRes
CRE_Tadv  = dR_dTadv * Tadv_en_loRes
CRE_RH700 = dR_dRH700 * RH700_en_loRes
CRE_w700  = dR_dw700 * w700_en_loRes
CRE_ws    = dR_dws * ws_en_loRes
CRE_TOT   = (CRE_SST.fillna(0) + CRE_EIS.fillna(0) + CRE_Tadv.fillna(0) + 
             CRE_ws.fillna(0) + CRE_RH700.fillna(0) + CRE_w700.fillna(0))

In [ ]:
def get_model_cre():
    # Set up subplots figure with PlateCarree projection
    fig, ax = plt.subplots(4, 2, figsize=(18,12), 
                           subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})
    fig.tight_layout(w_pad=6,h_pad=5)
    
    # Title list
    enso_cre_title = [['SST','EIS'],
                      ['Tadv','RH700'],
                      ['w700','WS'],
                      ['ALL CCFs','none']]
    
    vmin1 = -2
    vmax1 = 2
    vmin2 = -0.5
    vmax2 = 0.5
    vmin3 = -0.05
    vmax3 = 0.05

    # Give each plot gridlines, coastlines, and block out continents
    for i in np.arange(0,4,1):
        for j in np.arange(0,2,1):
            ax[i,j].gridlines()
            ax[i,j].coastlines()
            ax[i,j].add_feature(cartopy.feature.LAND,zorder=100,facecolor='k',edgecolor='k')

    # SST CRE
    CRE_SST.sel(nino34='nino').plot(ax=ax[0,0],transform=ccrs.PlateCarree(),
                                    cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                    vmin=vmin1,vmax=vmax1)
    # EIS
    CRE_EIS.sel(nino34='nino').plot(ax=ax[0,1],transform=ccrs.PlateCarree(),
                                    cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                    vmin=vmin1,vmax=vmax1)
    # Tadv
    CRE_Tadv.sel(nino34='nino').plot(ax=ax[1,0],transform=ccrs.PlateCarree(),
                                     cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                     vmin=vmin1,vmax=vmax1)
    # RH700
    CRE_RH700.sel(nino34='nino').plot(ax=ax[1,1],transform=ccrs.PlateCarree(),
                                      cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                      vmin=-0.2,vmax=0.2)
    # w700
    CRE_w700.sel(nino34='nino').plot(ax=ax[2,0],transform=ccrs.PlateCarree(),
                                     cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                     vmin=vmin3,vmax=vmax3)
    # ws
    CRE_ws.sel(nino34='nino').plot(ax=ax[2,1],transform=ccrs.PlateCarree(),
                                   cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                   vmin=vmin2,vmax=vmax2)
    # total
    CRE_TOT.sel(nino34='nino').plot(ax=ax[3,0],transform=ccrs.PlateCarree(),
                                    cmap=plt.get_cmap('RdBu').reversed(),cbar_kwargs={'shrink': 0.9,'label': 'W/m**2'},
                                    vmin=vmin1,vmax=vmax1)
    
    ax[3,1].set_visible(False)
    
    # Give plots their titles
    for i in np.arange(0,4,1):
        for j in np.arange(0,2,1):
            ax[i,j].set_title('El Nino Low Cloud CRE Anomaly - Contribution From ' + enso_cre_title[i][j])